In [6]:
import yfinance as yf
import pandas as pd

In [60]:
spx = yf.Ticker("SP=F")
gold = yf.Ticker("GC=F")

spx_df = pd.DataFrame(spx.history(period="10y"))
gold_df = pd.DataFrame(gold.history(period="10y"))

# pd.concat([spx["Close"], gold["Close"]], axis=1)
pd.DataFrame({"SPX": spx.history(period="10y"), "Gold": gold.history(period="10y")}, index=["SPX", ])

ValueError: If using all scalar values, you must pass an index

In [28]:
!jupyter nbconvert --to script *.ipynb

[NbConvertApp] Converting notebook api_test.ipynb to script
[NbConvertApp] Writing 74 bytes to api_test.txt
